In [298]:
'''this script takes a "history" frames reshapes it and merges it with pit stop

the actual data will be stored in 'raw_data' not in '2016race0'

the race_id (which is currently hard coded to 0) 
will be used with the ref_df dataframe to link it to race info (year,progressive number,track)

'''

'this script takes a "history" frames reshapes it and merges it with pit stop\n\nthe actual data will be stored in \'raw_data\' not in \'2016race0\'\n\nthe race_id (which is currently hard coded to 0) \nwill be used with the ref_df dataframe to link it to race info (year,progressive number,track)\n\n'

In [581]:
#import tabula
import pandas as pd
import os
import numpy as np
import re
import datetime

In [566]:
def find_divisorz(x): 
    if pd.notna(x):
        if 'NO' in x:
            #print(x,'is a special row')
            return True
        else:
            return False
    else: return True


def split_PIT(x,starters):
#     print(type(x['NO']))
    if x['NO'] not in starters:
        print('x non in starters')
        if pd.notna(x['NO']):
            print('x not null')
#        if x['NO'].notnull:
            if x['NO'][:-4] in starters:
                print('correcting PIT')
                return pd.Series([x['NO'][:-4],'PIT',x['TIME'],x['LAP']],index = ["NO","GAP","TIME","LAP"])
            elif x['NO'][:-6] in starters:
                print('correcting 1 LAP')
                return pd.Series([x['NO'][:-6],x['NO'][-5:],x['TIME'],x['LAP']],index = ["NO","GAP","TIME","LAP"])                
            else:
                print('trashing it')
                return pd.Series([None, None, None, None],index = ["NO","GAP","TIME","LAP"])
        else:
            print('it is a nan')
            return pd.Series([None, None, None, None],index = ["NO","GAP","TIME","LAP"])
    else:
        print('proper')
        return pd.Series(x,index = ["NO","GAP","TIME","LAP"])
        #return [x['NO'],x['GAP'],x['TIME'],x['LAP']]

def split_TIMES(x,starters):
    #print(x)
    if x['NO'] not in starters:
#         print('trashing it')
        return pd.Series([None, None, None, None],index = ["NO","GAP","TIME","LAP"])
    elif pd.isna(x['GAP']):
#         print('it is (hopefully) the leader')        
#         print(type(x),type(pd.Series([x['NO'],0,x['TIME'],x['LAP']])))
#         print(len(x),len(pd.Series([x['NO'],0,x['TIME'],x['LAP']])))
        return pd.Series([x['NO'],0,x['TIME'],x['LAP']],index = ["NO","GAP","TIME","LAP"])
    elif (' ' in str(x['GAP'])) & ('LAP' not in str(x['GAP'])):
#         print('correcting it (2)')
        timez = x['GAP'].split()
        return pd.Series([x['NO'],timez[0],timez[1],x['LAP']],index = ["NO","GAP","TIME","LAP"])
    elif pd.isna(x['TIME']):
#         print('it is a nan')        
        return pd.Series([x['NO'],0,x['GAP'],x['LAP']],index = ["NO","GAP","TIME","LAP"])
    else:
#         print('proper')
        return x

In [567]:
#df = history_chart_2_df
# d = {}
# d['df'] = pd.read_csv('2016race0/history_chart_2_df.csv')
# df = pd.read_csv('2016race0/history_chart_2_df.csv')
df = pd.read_csv('race_frames/history_chart_8_df.csv')
# d['pit_stop_df'] = pd.read_csv('2016race0/pit_stop_summary_6_df.csv')
#pit_stop_df = pd.read_csv('2016race0/pit_stop_summary_6_df.csv')
pit_stop_df = pd.read_csv('race_frames/pit_stops_8_df.csv')

In [568]:
df.columns=['Unnamed: 0', 'NO.0', 'GAP.0', 'TIME.0', 'NO.1', 'GAP.1', 'TIME.1', 'NO.2',
       'GAP.2', 'TIME.2', 'NO.3', 'GAP.3', 'TIME.3', 'NO.4', 'GAP.4',
       'TIME.4']
# df.rename(columns = {'NO':'NO.0','GAP':'GAP.0','TIME':'TIME.0'},inplace=True)

In [569]:
divisorz = df.loc[df['NO.0'].apply(find_divisorz)].index;
divisorz;

In [570]:
# slicing the history dataframe horizontally
previous_divisor = -1
d={}
for _id,_val in enumerate(divisorz):
#     print (_id,_val)
    d['df_'+str(_id)] = df.loc[previous_divisor+1:_val-1]
    previous_divisor = _val
    div_counter = _id
last_id = div_counter + 1
d['df_'+str(last_id)] = df.loc[previous_divisor+1:]

In [571]:
# d1['df_lap_'+str(1)] = d['df_'+str(1)][['NO.'+str(0),'GAP.'+str(0),'TIME.'+str(0)]]
# d1['df_lap_'+str(1)]['lap'] = str(1)
# # uniforming column names
# #             exec('df_lap_'+str(1)+'.columns = ["NO","GAP","TIME","LAP"]')
# d1['df_lap_'+str(1)].columns = ["NO","GAP","TIME","LAP"]
# # putting 0 were 'GAP' is none (!!WORKAROUND!!the first row of a frame appear to behave differently than 
# # other rows resulting in adding new columns to the data in an incompatible way. 
# # considering a bug report to pandas developers
# #             exec('df_lap_'+str(1)+'["GAP"].fillna(0,inplace=True)')
# d1['df_lap_'+str(1)]["GAP"].fillna(0,inplace=True)
# # removing empty rows (where racing number is empty)
# #             exec('df_lap_'+str(1)+' = df_lap_'+str(1)+'[df_lap_'+str(1)+'["NO"].notnull()]' )
# d1['df_lap_'+str(1)] = d1['df_lap_'+str(1)][d1['df_lap_'+str(1)]["NO"].notnull()]
# # solving the problem of 'PIT' value being collected in the 'NO' field
# #             exec('df_lap_'+str(1)+' = df_lap_'+str(1)+'.apply(split_PIT, axis =1)')
# #             exec(d1['df_lap_'+str(1)]+' = df_lap_'+str(1)+'.apply(split_PIT, axis =1)')
# # d1['df_lap_'+str(1)] = d1['df_lap_'+str(1)].transform(split_PIT, axis =1)
# # #             # putting back the right names
# # # #             exec('df_lap_'+str(1)+'.columns = ["NO","GAP","TIME","LAP"]')
# # #             d1['df_lap_'+str(1)].columns = ["NO","GAP","TIME","LAP"]
# # #             # removing empty rows (where racing number is empty).
# # # #             exec('df_lap_'+str(1)+' = df_lap_'+str(1)+'[df_lap_'+str(1)+'["NO"].notnull()]' )            
# # d1['df_lap_'+str(1)] = d1['df_lap_'+str(1)][d1['df_lap_'+str(1)]["NO"].notnull()]
# # #             # solving the problem of 'NO' and 'GAP' fields merging
# # # #             exec('df_lap_'+str(1)+' = df_lap_'+str(1)+'.apply(split_TIMES, axis =1)')
# # d1['df_lap_'+str(1)] = d1['df_lap_'+str(1)].apply(split_TIMES, axis =1)
# # #             # putting back the right names
# # # #             exec('df_lap_'+str(1)+'.columns = ["NO","GAP","TIME","LAP"]')
# # d1['df_lap_'+str(1)].columns = ["NO","GAP","TIME","LAP"]
# # #             # removing empty rows (where racing number is empty).
# # # #             exec('df_lap_'+str(1)+' = df_lap_'+str(1)+'[df_lap_'+str(1)+'["NO"].notnull()]' )            
# # d1['df_lap_'+str(1)] = d1['df_lap_'+str(1)][d1['df_lap_'+str(1)]["NO"].notnull()]

In [572]:
# d1['df_lap_1']

In [573]:
starters = d['df_1']['NO.0'].unique()

In [578]:
d1={}
for j in range(len(divisorz)+1):
#     print('j is ', j)
    for i in [0,1,2,3,4]:
#         print('i is ', i)
#         print('doing lap',j*5+(i+1))
        if j*5+(i+1):
            # Take the horizontal slices and slice them vertically into laps 
            d1['df_lap_'+str(j*5+(i+1))] = d['df_'+str(j)][['NO.'+str(i),'GAP.'+str(i),'TIME.'+str(i)]]
            # add lap feature 
            d1['df_lap_'+str(j*5+(i+1))]['lap'] = str(j*5+(i+1))
            # uniforming column names
            d1['df_lap_'+str(j*5+(i+1))].columns = ["NO","GAP","TIME","LAP"]
            # putting 0 were 'GAP' is none (!!WORKAROUND!!the first row of a frame appear to behave differently than 
            # other rows resulting in adding new columns to the data in an incompatible way. 
            # considering a bug report to pandas developers
            d1['df_lap_'+str(j*5+(i+1))]["GAP"].fillna(0,inplace=True)
            # removing empty rows (where racing number is empty)
            d1['df_lap_'+str(j*5+(i+1))] = d1['df_lap_'+str(j*5+(i+1))][d1['df_lap_'+str(j*5+(i+1))]["NO"].notnull()]
            # solving the problem of 'PIT' value being collected in the 'NO' field
#             d1['df_lap_'+str(j*5+(i+1))] = d1['df_lap_'+str(j*5+(i+1))].transform(lambda x: split_NO(x,starters), axis =1)
            d1['df_lap_'+str(j*5+(i+1))] = d1['df_lap_'+str(j*5+(i+1))].transform(lambda x: split_PIT(x,starters_2016), axis =1)
            # putting back the right names
            d1['df_lap_'+str(j*5+(i+1))].columns = ["NO","GAP","TIME","LAP"]
#             # removing empty rows (where racing number is empty).
            d1['df_lap_'+str(j*5+(i+1))] = d1['df_lap_'+str(j*5+(i+1))][d1['df_lap_'+str(j*5+(i+1))]["NO"].notnull()]
#             # solving the problem of 'NO' and 'GAP' fields merging
            d1['df_lap_'+str(j*5+(i+1))] = d1['df_lap_'+str(j*5+(i+1))].transform(lambda x: split_TIMES(x,starters_2016), axis =1)
#             # putting back the right names
            d1['df_lap_'+str(j*5+(i+1))].columns = ["NO","GAP","TIME","LAP"]
#             # removing empty rows (where racing number is empty).
            d1['df_lap_'+str(j*5+(i+1))] = d1['df_lap_'+str(j*5+(i+1))][d1['df_lap_'+str(j*5+(i+1))]["NO"].notnull()]

In [575]:
laps_dfs=[]
for j in range(len(divisorz)+1):
    for i in [0,1,2,3,4]:
        laps_dfs.append(d1['df_lap_'+str(j*5+(i+1))])

In [576]:
race_df = pd.concat(laps_dfs,ignore_index=True)

race_df['race_id'] = 0

race_df.drop(race_df.index[race_df['NO'].isna()],axis=0, inplace=True)

race_df['NO'] = race_df['NO'].astype('int64')

race_df['LAP'] = race_df['LAP'].astype('int64')

#race_df['GAP'].fillna(0,inplace=True)

In [ ]:
###checking times
_mask = race_df['TIME'].apply(check_times)#.values[0]#.split(' ')[0:2]
###correcting only problem
race_df['TIME'][_mask] = '2:11.365'
race_df['GAP'][_mask] = '1 LAP'

In [315]:
starters = starters[pd.notna(starters)].astype('int64')
finishers = race_df[race_df['LAP'] == max(race_df['LAP'])]['NO'].unique()
finishers = finishers[pd.notna(finishers)]

In [316]:
#pit_stop_df = pit_stop_summary_6_df

pit_stop_df = pit_stop_df[pit_stop_df['NO']!='NO']

pit_stop_df.rename(columns = {'S  T  OP':'STOP'},inplace=True)

pit_stop_df['NO'] = pit_stop_df['NO'].astype('int64')

pit_stop_df['LAP'] = pit_stop_df['LAP'].astype('int64')

In [317]:
# new_df = race_df.merge(pit_stop_df, on = ['NO','LAP'], how ='outer')
# new_df.drop(columns = ['Unnamed: 0','Unnamed: 6'], inplace=True)

In [562]:
# new_df['TIMESECS'] = new_df['TIME'].apply(lambda x: int(x.split(':')[0])*60 + float(x.split(':')[1]))
# new_df['TIME64'] = new_df['TIME'].apply(lambda x: datetime.datetime.strptime(x, "%M:%S.%f"))

In [48]:
# race_df.set_index(['NO','LAP'], inplace=True)
# pit_stop_df.set_index(['NO','LAP'], inplace=True)
# new_df = race_df.merge(pit_stop_df, on = ['NO','LAP'], how ='outer')


In [580]:
race_level_df = pit_stop_df[['STOP','NO']][pit_stop_df['NO'].isin(finishers)].groupby('NO').max().reset_index()
race_level_df['race_id'] = 0
#pit_stop_df[['STOP','NO']].groupby('NO').max()

KeyError: "['STOP'] not in index"

In [ ]:
ref_df = pd.read_csv('which_race.csv', header=0)

In [ ]:
race_level_df = race_level_df.merge(ref_df.reset_index(), on = ['race_id'], how ='inner')

race_level_df = race_level_df.merge(ref_df, on = ['race_id'], how ='inner')

In [207]:
pit_stop_0_df = pd.read_csv('race_frames/pit_stops_1_df.csv')

In [537]:
# df_names = []
# for _pdf in os.listdir('2016race0'):
#     if (_pdf.lower().endswith(('.pdf'))) & ('jpg' not in _pdf.lower()):
#         _name = _pdf[7:-9]+'_df'
# #         print(_pdf)
# #         print(_name)
#         df_names.append(_name)
#         exec( _name+" = tabula.read_pdf('2016race0/'+_pdf,pages='all')")
# #        exec(_name+'["race_id"] = 0')
#         exec(_name+".to_csv('2016race0/'+_name+'.csv')")

# df_names = []
# for _csv in os.listdir('2016race0'):
#     if _csv.lower().endswith(('.csv')):
#         _name = _csv[:-4]
# #         print(_csv)
# #         print(_name)
#         df_names.append(_name)
#         exec(_name+" = pd.read_csv('2016race0/'+_name+'.csv')")
# #        exec(_name+"['race_id'] = 0")

In [560]:
starters_2016 = np.array(["5","7","11","27","8","21","14","47","22","6","44","88","31","94","3","26","33","20","30","9","12","33","26","55","19","77"])

In [579]:
starters_2017 = ["5","7","11","31","8","20","2","14","22","44","77","3","33","27","30","55","9","36","94","26","10","28","39","55","26","10","18","19","40"]